# Assignment 3


- Student Name: Karina Jonina
- Student ID: c00278440

- Module Name: Programming for Data Scientists
- Module Code: PROGC5201
- Module Leader: Paul Barry

- Course Name: 		Master of Science in Data Science
- Course Code:		 CW_KCDAR_M Y5

- Due Date: 10th January 2022 9:00am

In [1]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

In [2]:
# gettting the CSV from the website
url = 'https://opendata-geohive.hub.arcgis.com/datasets/d8eb52d56273413b84b0187a4e9117be_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D.'

df = pd.read_csv(url, parse_dates = ["Date"])

In [3]:
df.head()

,X,Y,Date,ConfirmedCovidCases,TotalConfirmedCovidCases,ConfirmedCovidDeaths,TotalCovidDeaths,StatisticsProfileDate,CovidCasesConfirmed,HospitalisedCovidCases,...,CommunityTransmission,CloseContact,TravelAbroad,FID,HospitalisedAged65to74,HospitalisedAged75to84,HospitalisedAged85up,Aged65to74,Aged75to84,Aged85up
0,-7.692596,53.288234,2020-02-29 00:00:00+00:00,1,1,0.0,0,2020/02/27 00:00:00+00,NaN,NaN,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,-7.692596,53.288234,2020-03-03 00:00:00+00:00,1,2,0.0,0,2020/03/01 00:00:00+00,NaN,NaN,...,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN
2,-7.692596,53.288234,2020-03-04 00:00:00+00:00,4,6,0.0,0,2020/03/02 00:00:00+00,1.0,0.0,...,0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
3,-7.692596,53.288234,2020-03-05 00:00:00+00:00,7,13,0.0,0,2020/03/03 00:00:00+00,2.0,1.0,...,0,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0
4,-7.692596,53.288234,2020-03-06 00:00:00+00:00,5,18,0.0,0,2020/03/04 00:00:00+00,5.0,4.0,...,0,0,0,5,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# examiningcolumnsand rows
df.shape

(644, 41)

In [5]:
# checking type for each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   X                            644 non-null    float64            
 1   Y                            644 non-null    float64            
 2   Date                         644 non-null    datetime64[ns, UTC]
 3   ConfirmedCovidCases          644 non-null    int64              
 4   TotalConfirmedCovidCases     644 non-null    int64              
 5   ConfirmedCovidDeaths         498 non-null    float64            
 6   TotalCovidDeaths             644 non-null    int64              
 7   StatisticsProfileDate        644 non-null    object             
 8   CovidCasesConfirmed          642 non-null    float64            
 9   HospitalisedCovidCases       642 non-null    float64            
 10  RequiringICUCovidCases       642 non-null    float

In [6]:
# examining the names of the columns
df.columns

Index(['X', 'Y', 'Date', 'ConfirmedCovidCases', 'TotalConfirmedCovidCases',
       'ConfirmedCovidDeaths', 'TotalCovidDeaths', 'StatisticsProfileDate',
       'CovidCasesConfirmed', 'HospitalisedCovidCases',
       'RequiringICUCovidCases', 'HealthcareWorkersCovidCases',
       'ClustersNotified', 'HospitalisedAged5', 'HospitalisedAged5to14',
       'HospitalisedAged15to24', 'HospitalisedAged25to34',
       'HospitalisedAged35to44', 'HospitalisedAged45to54',
       'HospitalisedAged55to64', 'Male', 'Female', 'Unknown', 'Aged1to4',
       'Aged5to14', 'Aged15to24', 'Aged25to34', 'Aged35to44', 'Aged45to54',
       'Aged55to64', 'Median_Age', 'CommunityTransmission', 'CloseContact',
       'TravelAbroad', 'FID', 'HospitalisedAged65to74',
       'HospitalisedAged75to84', 'HospitalisedAged85up', 'Aged65to74',
       'Aged75to84', 'Aged85up'],
      dtype='object')

In [7]:
# changing StatisticsProfileDate to datetime 
df["StatisticsProfileDate"] #= pd.to_datetime(df["StatisticsProfileDate"], errors='ignore')

0      2020/02/27 00:00:00+00
1      2020/03/01 00:00:00+00
2      2020/03/02 00:00:00+00
3      2020/03/03 00:00:00+00
4      2020/03/04 00:00:00+00
                ...          
639    2021/11/29 00:00:00+00
640    2021/11/30 00:00:00+00
641    2021/12/01 00:00:00+00
642    2021/12/01 00:00:00+00
643    2021/12/01 00:00:00+00
Name: StatisticsProfileDate, Length: 644, dtype: object

## Examining location
Below exploration showed that there are no unique locations in the dataset and therefore, redundant. 

In [8]:
df['X'].unique()

array([-7.692596])

In [9]:
df['Y'].unique()

array([53.288234])

## Examining date
Below exploration showed that there are are unique dates in the dataset. 

In [10]:
df['Date'].unique()

<DatetimeArray>
['2020-02-29 00:00:00+00:00', '2020-03-03 00:00:00+00:00',
 '2020-03-04 00:00:00+00:00', '2020-03-05 00:00:00+00:00',
 '2020-03-06 00:00:00+00:00', '2020-03-07 00:00:00+00:00',
 '2020-03-08 00:00:00+00:00', '2020-03-09 00:00:00+00:00',
 '2020-03-10 00:00:00+00:00', '2020-03-11 00:00:00+00:00',
 ...
 '2021-11-26 00:00:00+00:00', '2021-11-27 00:00:00+00:00',
 '2021-11-28 00:00:00+00:00', '2021-11-29 00:00:00+00:00',
 '2021-11-30 00:00:00+00:00', '2021-12-01 00:00:00+00:00',
 '2021-12-02 00:00:00+00:00', '2021-12-03 00:00:00+00:00',
 '2021-12-04 00:00:00+00:00', '2021-12-05 00:00:00+00:00']
Length: 644, dtype: datetime64[ns, UTC]

In [11]:
df.head()

,X,Y,Date,ConfirmedCovidCases,TotalConfirmedCovidCases,ConfirmedCovidDeaths,TotalCovidDeaths,StatisticsProfileDate,CovidCasesConfirmed,HospitalisedCovidCases,...,CommunityTransmission,CloseContact,TravelAbroad,FID,HospitalisedAged65to74,HospitalisedAged75to84,HospitalisedAged85up,Aged65to74,Aged75to84,Aged85up
0,-7.692596,53.288234,2020-02-29 00:00:00+00:00,1,1,0.0,0,2020/02/27 00:00:00+00,NaN,NaN,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,-7.692596,53.288234,2020-03-03 00:00:00+00:00,1,2,0.0,0,2020/03/01 00:00:00+00,NaN,NaN,...,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN
2,-7.692596,53.288234,2020-03-04 00:00:00+00:00,4,6,0.0,0,2020/03/02 00:00:00+00,1.0,0.0,...,0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
3,-7.692596,53.288234,2020-03-05 00:00:00+00:00,7,13,0.0,0,2020/03/03 00:00:00+00,2.0,1.0,...,0,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0
4,-7.692596,53.288234,2020-03-06 00:00:00+00:00,5,18,0.0,0,2020/03/04 00:00:00+00,5.0,4.0,...,0,0,0,5,0.0,0.0,0.0,0.0,0.0,0.0


## Melting 

In [12]:
# Melting
id_vars = ["X","Y","Date"]

df = pd.melt(frame=df,id_vars=id_vars, var_name="column", value_name="cases")

df.head(10)


,X,Y,Date,column,cases
0,-7.692596,53.288234,2020-02-29 00:00:00+00:00,ConfirmedCovidCases,1
1,-7.692596,53.288234,2020-03-03 00:00:00+00:00,ConfirmedCovidCases,1
2,-7.692596,53.288234,2020-03-04 00:00:00+00:00,ConfirmedCovidCases,4
3,-7.692596,53.288234,2020-03-05 00:00:00+00:00,ConfirmedCovidCases,7
4,-7.692596,53.288234,2020-03-06 00:00:00+00:00,ConfirmedCovidCases,5
5,-7.692596,53.288234,2020-03-07 00:00:00+00:00,ConfirmedCovidCases,1
6,-7.692596,53.288234,2020-03-08 00:00:00+00:00,ConfirmedCovidCases,2
7,-7.692596,53.288234,2020-03-09 00:00:00+00:00,ConfirmedCovidCases,3
8,-7.692596,53.288234,2020-03-10 00:00:00+00:00,ConfirmedCovidCases,10
9,-7.692596,53.288234,2020-03-11 00:00:00+00:00,ConfirmedCovidCases,9


### Dropping cases

In [13]:
df = df[[
    "Date",
    "column",
    "cases"
    
]]

In [14]:
df.head()

,Date,column,cases
0,2020-02-29 00:00:00+00:00,ConfirmedCovidCases,1
1,2020-03-03 00:00:00+00:00,ConfirmedCovidCases,1
2,2020-03-04 00:00:00+00:00,ConfirmedCovidCases,4
3,2020-03-05 00:00:00+00:00,ConfirmedCovidCases,7
4,2020-03-06 00:00:00+00:00,ConfirmedCovidCases,5


# Hospitalisations

In [15]:
hospitalised_df = df[df['column'].str.contains("Hospitalised")]
hospitalised_df.head()

,Date,column,cases
3864,2020-02-29 00:00:00+00:00,HospitalisedCovidCases,NaN
3865,2020-03-03 00:00:00+00:00,HospitalisedCovidCases,NaN
3866,2020-03-04 00:00:00+00:00,HospitalisedCovidCases,0.0
3867,2020-03-05 00:00:00+00:00,HospitalisedCovidCases,1.0
3868,2020-03-06 00:00:00+00:00,HospitalisedCovidCases,4.0


In [16]:
hospitalised_df['column'].unique()

array(['HospitalisedCovidCases', 'HospitalisedAged5',
       'HospitalisedAged5to14', 'HospitalisedAged15to24',
       'HospitalisedAged25to34', 'HospitalisedAged35to44',
       'HospitalisedAged45to54', 'HospitalisedAged55to64',
       'HospitalisedAged65to74', 'HospitalisedAged75to84',
       'HospitalisedAged85up'], dtype=object)

In [17]:
#hospitalised_df.iloc[:,1] = hospitalised_df.iloc[:,1].str.lstrip("Hospitalised")

In [18]:
hospitalised_df['column'].tail()

22535    HospitalisedAged85up
22536    HospitalisedAged85up
22537    HospitalisedAged85up
22538    HospitalisedAged85up
22539    HospitalisedAged85up
Name: column, dtype: object

In [19]:
hospitalised_df['column'].unique()

array(['HospitalisedCovidCases', 'HospitalisedAged5',
       'HospitalisedAged5to14', 'HospitalisedAged15to24',
       'HospitalisedAged25to34', 'HospitalisedAged35to44',
       'HospitalisedAged45to54', 'HospitalisedAged55to64',
       'HospitalisedAged65to74', 'HospitalisedAged75to84',
       'HospitalisedAged85up'], dtype=object)

In [20]:
test = "Aged5to14"

match = re.search("(\D+)(\d+)(\D{2})(\d{2})", test)
print(match[0])
print(match[1]) # Aged
print(match[2]) # 5
print(match[3]) # to
print(match[4]) # 14
print()

Aged5to14
Aged
5
to
14



In [21]:
hospitalised_df["column"]

3864     HospitalisedCovidCases
3865     HospitalisedCovidCases
3866     HospitalisedCovidCases
3867     HospitalisedCovidCases
3868     HospitalisedCovidCases
                  ...          
22535      HospitalisedAged85up
22536      HospitalisedAged85up
22537      HospitalisedAged85up
22538      HospitalisedAged85up
22539      HospitalisedAged85up
Name: column, Length: 7084, dtype: object

In [22]:
tmp_df = hospitalised_df["column"].str.extract("(\D+)(\d+)(\D{2})(\d{2})", expand=False)    


# Name columns
tmp_df.columns = ["Aged", "age_lower","to", "age_upper"]

# Create `age`column based on `age_lower` and `age_upper`
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

tmp_df.head(15)

,Aged,age_lower,to,age_upper,age
3864,NaN,NaN,NaN,NaN,NaN
3865,NaN,NaN,NaN,NaN,NaN
3866,NaN,NaN,NaN,NaN,NaN
3867,NaN,NaN,NaN,NaN,NaN
3868,NaN,NaN,NaN,NaN,NaN
3869,NaN,NaN,NaN,NaN,NaN
3870,NaN,NaN,NaN,NaN,NaN
3871,NaN,NaN,NaN,NaN,NaN
3872,NaN,NaN,NaN,NaN,NaN
3873,NaN,NaN,NaN,NaN,NaN


In [23]:
# Merge 
hospitalised_df = pd.concat([hospitalised_df, tmp_df], axis=1)

# Drop unnecessary columns and rows
hospitalised_df = hospitalised_df.drop(['column',"age_lower","to","age_upper" ], axis=1)
hospitalised_df = hospitalised_df.dropna()
hospitalised_df = hospitalised_df.sort_values(ascending = True,by = ["Date"])
hospitalised_df["cases"] = hospitalised_df["cases"].astype(int)



In [24]:

hospitalised_df["Aged"] = hospitalised_df["Aged"].str.rstrip("Aged")
hospitalised_df.head(25)

,Date,cases,Aged,age
7086,2020-03-04 00:00:00+00:00,0,Hospitalis,5-14
8374,2020-03-04 00:00:00+00:00,0,Hospitalis,25-34
9018,2020-03-04 00:00:00+00:00,0,Hospitalis,35-44
9662,2020-03-04 00:00:00+00:00,0,Hospitalis,45-54
10306,2020-03-04 00:00:00+00:00,0,Hospitalis,55-64
20610,2020-03-04 00:00:00+00:00,0,Hospitalis,65-74
21254,2020-03-04 00:00:00+00:00,0,Hospitalis,75-84
7730,2020-03-04 00:00:00+00:00,0,Hospitalis,15-24
8375,2020-03-05 00:00:00+00:00,0,Hospitalis,25-34
9663,2020-03-05 00:00:00+00:00,1,Hospitalis,45-54


# Ages

In [25]:
aged_df = df[df["column"].str.contains("^Aged")]

In [26]:
aged_df.head()

,Date,column,cases
12880,2020-02-29 00:00:00+00:00,Aged1to4,NaN
12881,2020-03-03 00:00:00+00:00,Aged1to4,NaN
12882,2020-03-04 00:00:00+00:00,Aged1to4,0.0
12883,2020-03-05 00:00:00+00:00,Aged1to4,0.0
12884,2020-03-06 00:00:00+00:00,Aged1to4,0.0


In [27]:
aged_df["column"].unique()

array(['Aged1to4', 'Aged5to14', 'Aged15to24', 'Aged25to34', 'Aged35to44',
       'Aged45to54', 'Aged55to64', 'Aged65to74', 'Aged75to84', 'Aged85up'],
      dtype=object)

In [28]:
test = "Aged1to4"

match = re.search("(\D+)(\d+)(\D{2})(\d)", test)
print(match[0])
print(match[1]) # Aged
print(match[2]) # number 
print(match[3]) # to
print(match[4]) # number
print()



test = "Aged35to44"

# incorrectly drops the 
match = re.search("(\D+)(\d+)(\D{2})(\d+)", test)
print(match[0])
print(match[1]) # Aged
print(match[2]) # number 
print(match[3]) # to
print(match[4]) # number
print()



Aged1to4
Aged
1
to
4

Aged35to44
Aged
35
to
44



In [29]:
tmp_df = aged_df["column"].str.extract("(\D+)(\d+)(\D{2})(\d+)", expand=False)    


# Name columns
tmp_df.columns = ["Aged", "age_lower","to", "age_upper"]

# Create `age`column based on `age_lower` and `age_upper`
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

tmp_df.head(15)

,Aged,age_lower,to,age_upper,age
12880,Aged,1,to,4,1-4
12881,Aged,1,to,4,1-4
12882,Aged,1,to,4,1-4
12883,Aged,1,to,4,1-4
12884,Aged,1,to,4,1-4
12885,Aged,1,to,4,1-4
12886,Aged,1,to,4,1-4
12887,Aged,1,to,4,1-4
12888,Aged,1,to,4,1-4
12889,Aged,1,to,4,1-4


In [30]:
# Merge 
aged_df = pd.concat([aged_df, tmp_df], axis=1)

# Drop unnecessary columns and rows
aged_df = aged_df.drop(['column',"age_lower","to","age_upper" ], axis=1)
aged_df = aged_df.dropna()
aged_df = aged_df.sort_values(ascending = True,by = ["Date"])
aged_df["cases"] = aged_df["cases"].astype(int)
aged_df.head(25)

,Date,cases,Aged,age
12882,2020-03-04 00:00:00+00:00,0,Aged,1-4
16746,2020-03-04 00:00:00+00:00,0,Aged,55-64
16102,2020-03-04 00:00:00+00:00,0,Aged,45-54
15458,2020-03-04 00:00:00+00:00,0,Aged,35-44
23186,2020-03-04 00:00:00+00:00,0,Aged,75-84
14170,2020-03-04 00:00:00+00:00,0,Aged,15-24
22542,2020-03-04 00:00:00+00:00,0,Aged,65-74
13526,2020-03-04 00:00:00+00:00,1,Aged,5-14
14814,2020-03-04 00:00:00+00:00,0,Aged,25-34
14815,2020-03-05 00:00:00+00:00,0,Aged,25-34


### Getting the final DF

In [32]:
# Merge 
df = pd.concat([df, aged_df])
df = pd.concat([df, hospitalised_df])

# Drop unnecessary columns and rows
df = df.sort_values(ascending = True,by = ["Date"])
df["cases"] = df["cases"].astype(int)
df.head(25)

ValueError: cannot convert float NaN to integer

In [ ]:
#df.to_csv(r"df.csv", index =  False)